In [1]:
import pandas as pd
import os
import calendar

os.chdir(os.getcwd()) 


In [2]:
data_input = pd.read_csv(r'archive (2)/weatherAUS.csv')

data_input.head()
data_input.shape

(145460, 11)

In [3]:
data_input['Date'] = pd.to_datetime(data_input['Date'],infer_datetime_format=True)

C:\Users\alan1\AppData\Local\Temp\ipykernel_6408\437029015.py:1: UserWarning: Parsing '13/12/2008' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  data_input['Date'] = pd.to_datetime(data_input['Date'],infer_datetime_format=True)
C:\Users\alan1\AppData\Local\Temp\ipykernel_6408\437029015.py:1: UserWarning: Parsing '14/12/2008' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  data_input['Date'] = pd.to_datetime(data_input['Date'],infer_datetime_format=True)
C:\Users\alan1\AppData\Local\Temp\ipykernel_6408\437029015.py:1: UserWarning: Parsing '15/12/2008' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  data_input['Date'] = pd.to_datetime(data_input['Date'],infer_datetime_format=True)
C:\Users\alan1\AppData\Local\Temp\ipykernel_6408\437029015.py:1: UserWarning: Parsing '16/12/2008' in DD/MM/YYYY format. Provide format or specify infe

In [4]:

state_mapping = pd.read_csv(r'archive (2)/weatherAUS- States Mapping.csv')

state_id_dict = {'New South Wales' : 0, 'Victoria' : 1, 'Queensland' : 2, 'South Australia' : 3, 'Western Australia' : 4, 'Tasmania' : 5, 'Northern Territory' : 6, 'Australian Capital Territory' : 7}
state_id_dict_reversed = { 0: 'New South Wales', 1 : 'Victoria' , 2 : 'Queensland', 3 : 'South Australia', 4 : 'Western Australia', 5 : 'Tasmania', 6 : 'Northern Territory', 7 : 'Australian Capital Territory'}


data_input_processed = data_input.merge(state_mapping,how='right', on='Location')
data_input_processed = data_input_processed.merge(state_mapping,how='right', on='Location', suffixes=[None, '_y'])

data_input_processed['id'] = data_input_processed['State_y'].map(state_id_dict)

data_input_processed = data_input_processed.drop(columns='State_y')
data_input_processed = data_input_processed.dropna(subset=['Date'])

data_input_processed['Year'] = data_input_processed['Date'].dt.year.astype(int)
data_input_processed['Month'] = data_input_processed['Date'].dt.month.astype(int)

data_input_processed.isnull().sum()

Date              0
Location          0
latitude          0
longitude         0
MinTemp        1419
MaxTemp        1203
Rainfall       3090
Humidity9am    2595
Humidity3pm    3784
Temp9am        1718
Temp3pm        2891
State             0
id                0
Year              0
Month             0
dtype: int64

In [5]:
data_input_processed.head()

,Date,Location,latitude,longitude,MinTemp,MaxTemp,Rainfall,Humidity9am,Humidity3pm,Temp9am,Temp3pm,State,id,Year,Month
0,2008-01-12,Albury,-36.08078,146.916473,13.4,22.9,0.6,71.0,22.0,16.9,21.8,New South Wales,0,2008,1
1,2008-02-12,Albury,-36.08078,146.916473,7.4,25.1,0.0,44.0,25.0,17.2,24.3,New South Wales,0,2008,2
2,2008-03-12,Albury,-36.08078,146.916473,12.9,25.7,0.0,38.0,30.0,21.0,23.2,New South Wales,0,2008,3
3,2008-04-12,Albury,-36.08078,146.916473,9.2,28.0,0.0,45.0,16.0,18.1,26.5,New South Wales,0,2008,4
4,2008-05-12,Albury,-36.08078,146.916473,17.5,32.3,1.0,82.0,33.0,17.8,29.7,New South Wales,0,2008,5


In [6]:
data_input_yearly_average = data_input_processed.groupby(by=['id', 'Year']).mean().round(2)

data_input_yearly_average = data_input_yearly_average.reset_index()
data_input_yearly_average['State'] = data_input_yearly_average['id'].map(state_id_dict_reversed)
data_input_yearly_average = data_input_yearly_average.drop(columns=['Month', 'latitude','longitude','Humidity9am','Humidity3pm','Temp9am','Temp3pm'])
data_input_yearly_average

,id,Year,MinTemp,MaxTemp,Rainfall,State
0,0,2008,14.31,23.19,2.84,New South Wales
1,0,2009,13.20,24.18,2.32,New South Wales
2,0,2010,13.10,23.18,2.88,New South Wales
3,0,2011,12.81,23.19,3.03,New South Wales
4,0,2012,11.80,22.82,2.81,New South Wales
...,...,...,...,...,...,...
76,7,2013,5.02,17.99,1.83,Australian Capital Territory
77,7,2014,6.03,18.35,1.99,Australian Capital Territory
78,7,2015,5.68,17.74,1.92,Australian Capital Territory
79,7,2016,6.44,17.92,2.80,Australian Capital Territory


In [7]:
data_input_yearly_average.to_csv('WeatherAUS - Yearly Average.csv')

# Line Graph
- two lines for max and min temp
- Separated by states

In [14]:
data_input_processed


data_input_monthly_yearly_average = data_input_processed.groupby(by=['id', 'Month','Year']).mean().round(2)

data_input_monthly_yearly_average = data_input_monthly_yearly_average.drop(columns=['latitude', 'longitude', 'Humidity9am', 'Humidity3pm','Temp9am','Temp3pm'])
data_input_monthly_yearly_average = data_input_monthly_yearly_average.reset_index()
data_input_monthly_yearly_average['State'] = data_input_monthly_yearly_average['id'].map(state_id_dict_reversed)
data_input_monthly_yearly_average['Month String'] = data_input_monthly_yearly_average['Month'].apply(lambda x: calendar.month_abbr[x])

data_input_monthly_yearly_average

,id,Month,Year,MinTemp,MaxTemp,Rainfall,State,Month String
0,0,1,2008,13.41,23.16,1.24,New South Wales,Jan
1,0,1,2009,17.02,29.26,1.66,New South Wales,Jan
2,0,1,2010,17.01,27.65,2.16,New South Wales,Jan
3,0,1,2011,17.45,28.52,1.25,New South Wales,Jan
4,0,1,2012,16.17,25.90,6.11,New South Wales,Jan
...,...,...,...,...,...,...,...,...
967,7,12,2013,9.37,24.12,1.63,Australian Capital Territory,Dec
968,7,12,2014,9.03,22.28,1.34,Australian Capital Territory,Dec
969,7,12,2015,8.82,22.49,1.94,Australian Capital Territory,Dec
970,7,12,2016,10.45,22.63,2.63,Australian Capital Territory,Dec


In [15]:
data_input_monthly_yearly_average.to_csv(r'WeatherAUS - Monthly and Yearly Average.csv')

In [10]:

data_input_monthly_average = data_input_processed.groupby(by=['id', 'Month']).mean().round(2)
data_input_monthly_average = data_input_monthly_average.drop(columns=['latitude', 'longitude', 'Humidity9am', 'Humidity3pm','Temp9am','Temp3pm', 'Year'])
data_input_monthly_average = data_input_monthly_average.reset_index()
data_input_monthly_average['State'] = data_input_monthly_average['id'].map(state_id_dict_reversed)
data_input_monthly_average['Month String'] = data_input_monthly_average['Month'].apply(lambda x: calendar.month_abbr[x])



data_input_monthly_average

,id,Month,MinTemp,MaxTemp,Rainfall,State,Month String
0,0,1,17.17,28.12,3.22,New South Wales,Jan
1,0,2,16.37,26.48,3.48,New South Wales,Feb
2,0,3,15.54,25.74,3.47,New South Wales,Mar
3,0,4,13.07,23.59,3.53,New South Wales,Apr
4,0,5,10.82,21.46,2.93,New South Wales,May
...,...,...,...,...,...,...,...
91,7,8,2.67,14.14,2.07,Australian Capital Territory,Aug
92,7,9,4.10,16.37,2.34,Australian Capital Territory,Sep
93,7,10,5.82,18.70,2.12,Australian Capital Territory,Oct
94,7,11,8.12,20.93,2.79,Australian Capital Territory,Nov


In [11]:
data_input_monthly_average.to_csv(r'WeatherAUS - Monthly Average.csv')

In [12]:
data_input_daily = data_input_processed.round(2)
data_input_daily = data_input_daily.drop(columns=['latitude', 'longitude', 'Humidity9am', 'Humidity3pm','Temp9am','Temp3pm', 'Location'])
data_input_daily = data_input_daily.reset_index()
data_input_daily['Date String'] = data_input_daily['Date'].astype(str)
data_input_daily 

,index,Date,MinTemp,MaxTemp,Rainfall,State,id,Year,Month,Date String
0,0,2008-01-12,13.4,22.9,0.6,New South Wales,0,2008,1,2008-01-12
1,1,2008-02-12,7.4,25.1,0.0,New South Wales,0,2008,2,2008-02-12
2,2,2008-03-12,12.9,25.7,0.0,New South Wales,0,2008,3,2008-03-12
3,3,2008-04-12,9.2,28.0,0.0,New South Wales,0,2008,4,2008-04-12
4,4,2008-05-12,17.5,32.3,1.0,New South Wales,0,2008,5,2008-05-12
...,...,...,...,...,...,...,...,...,...,...
136213,136213,2017-06-21,2.8,23.4,0.0,Northern Territory,6,2017,6,2017-06-21
136214,136214,2017-06-22,3.6,25.3,0.0,Northern Territory,6,2017,6,2017-06-22
136215,136215,2017-06-23,5.4,26.9,0.0,Northern Territory,6,2017,6,2017-06-23
136216,136216,2017-06-24,7.8,27.0,0.0,Northern Territory,6,2017,6,2017-06-24


In [13]:
data_input_daily.to_csv(r'WeatherAUS - Daily.csv')

# Ideas
- Fix time line in Aus graph

# Story
- Intro
- Aus graph - Which states are hotter than others?
- daily data - Seasonal effects, average over time
- TS (All States) - break down seaonal effect, compare states, declutter the data from daily data
- Make SC - Max Temp vs Min Temp  xxxxx
- Rainfall vs Temp (Area) + SC - comparing rainfall to temperature, exaploratory
- 